In [60]:
import pandas as pd
import random
import numpy as np
import torch

In [61]:
data = pd.read_csv('cleaned_data.csv')
data.rename(columns={' amazement':'amazement', 
                     ' solemnity':'solemnity', 
                     ' tenderness':'tenderness', 
                     ' nostalgia':'nostalgia',
                     ' calmness':'calmness', 
                     ' power': 'power', 
                     ' joyful_activation':'joyful_activation', 
                     ' tension':'tension', 
                     ' sadness':'sadness',
                     ' mood':'mood', 
                     ' age':'age'}, inplace=True)
data = data.drop(['Unnamed: 0', 'track id', ' liked', ' disliked', ' gender', ' mother tongue', 'merge_key', 'Title',
       'Artist','file_id'], axis=1)

In [62]:
genres = ['Pop', 'Rock', 'Jazz', 'Classical', 'Electronic', 'Hip-hop'] 
bpm_range = (60, 200)
listening_frequencies = ['hourly', 'daily', 'weekly', 'monthly']

data['avg_bpm_listened'] = [random.randint(bpm_range[0], bpm_range[1]) for _ in range(len(data))]
data['most_listened_genre'] = [random.choice(genres) for _ in range(len(data))]
data['listening_frequency'] = [random.choice(listening_frequencies) for _ in range(len(data))]

In [63]:
input_vector_columns = [
    'amazement', 'solemnity', 'tenderness', 'nostalgia', 'calmness', 'power',
    'joyful_activation', 'tension', 'sadness', 'mood', 'age', 
    'avg_bpm_listened', 'most_listened_genre', 'listening_frequency'
]
input_vector = data[input_vector_columns]

In [64]:
# For 'genre' and 'key', we convert to a long tensor of class indices, not one-hot.
# For 'bpm', we keep the normalized values as they are used for regression.

genre = data['Genre']
key = data['Key']
bpm = data['BPM']

# Assuming 'genre' and 'key' are categorical data, you convert them to corresponding indices.
# This process is known as label encoding. The LabelEncoder from sklearn can help with this.

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Label encode genre and key
genre_encoder = LabelEncoder()
genre_labels = genre_encoder.fit_transform(genre)

key_encoder = LabelEncoder()
key_labels = key_encoder.fit_transform(key)

# Normalize bpm as it's a continuous value
bpm_values = bpm.values.reshape(-1, 1)  # Reshaping to conform to the scaler's requirements
scaler = MinMaxScaler()
bpm_normalized = scaler.fit_transform(bpm_values)

# Convert all to PyTorch tensors
genre_tensor = torch.tensor(genre_labels, dtype=torch.long)  # Labels are indices, so they should be long tensors
key_tensor = torch.tensor(key_labels, dtype=torch.long)  # Same for these labels
bpm_tensor = torch.tensor(bpm_normalized, dtype=torch.float32)  # This remains a float tensor

targets = {
    "genre": genre_tensor,
    "key": key_tensor,
    "bpm": bpm_tensor
}


In [65]:
binary_features = input_vector[['amazement', 'solemnity', 'tenderness', 'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension', 'sadness']]
categorical_features = input_vector[['most_listened_genre', 'listening_frequency']]
continuous_features = input_vector[['mood', 'age', 'avg_bpm_listened']]

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
continuous_normalized = scaler.fit_transform(continuous_features)

binary_df = pd.DataFrame(binary_features, dtype=np.float32)
continuous_normalized_df = pd.DataFrame(continuous_normalized, columns=continuous_features.columns)

from sklearn.preprocessing import LabelEncoder

label_encoders = {} 
categorical_features_encoded = categorical_features.copy() 

for column in categorical_features.columns:
    le = LabelEncoder()
    categorical_features_encoded[column] = le.fit_transform(categorical_features[column])
    label_encoders[column] = le  

# Combine all the features into a single DataFrame
input_vector_processed = pd.concat([binary_df, continuous_normalized_df, categorical_features_encoded], axis=1)

# Convert the combined DataFrame into a tensor
input_tensor = torch.tensor(input_vector_processed.values, dtype=torch.float32)

In [78]:
import torch.nn as nn
import torch.nn.functional as F

class MultiOutputModel(nn.Module):
    def __init__(self, num_binary, num_continuous, num_genre_labels, num_key_labels):
        super(MultiOutputModel, self).__init__()

        # Total number of features after concatenation
        total_num_features = num_binary + num_continuous + 2

        # Common layers
        self.fc1 = nn.Linear(total_num_features, 64)
        self.fc2 = nn.Linear(64, 32)

        # Task-specific layers
        self.fc_genre = nn.Linear(32, num_genre_labels)
        self.fc_key = nn.Linear(32, num_key_labels)
        self.fc_bpm = nn.Linear(32, 1)

    def forward(self, binary_data, continuous_data, categorical_data):
        # Concatenate all features into a single large input vector
        x = torch.cat([binary_data, continuous_data, categorical_data], dim=1)  # Concatenate along feature dimension

        # Common layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # Task-specific layers
        out_genre = self.fc_genre(x)
        out_key = self.fc_key(x)
        out_bpm = self.fc_bpm(x)

        # Dictionary of outputs
        outputs = {
            "genre": out_genre,
            "key": out_key,
            "bpm": out_bpm
        }
        return outputs

In [79]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch

binary_tensor = torch.tensor(binary_df.values, dtype=torch.float32)
non_categorical_tensor = torch.tensor(continuous_normalized_df.values, dtype=torch.float32)
categorical_tensor = torch.tensor(categorical_features_encoded.values, dtype=torch.long)

class CustomDataset(Dataset):
    def __init__(self, inputs_binary, inputs_non_cat, inputs_cat, targets):
        self.inputs_binary = inputs_binary
        self.inputs_non_cat = inputs_non_cat
        self.inputs_cat = inputs_cat
        self.targets = targets

    def __len__(self):
        return len(self.targets['genre'])  # assuming length is same for all targets

    def __getitem__(self, idx):
        binary = self.inputs_binary[idx]
        non_cat = self.inputs_non_cat[idx]
        cat = self.inputs_cat[idx]
        target = {key: val[idx] for key, val in self.targets.items()}
        return binary, non_cat, cat, target

# Combine all inputs and targets into a single dataset
full_dataset = CustomDataset(
    inputs_binary=binary_tensor,
    inputs_non_cat=non_categorical_tensor,
    inputs_cat=categorical_tensor,
    targets=targets
)

# Define a size for your validation set
val_size = int(len(full_dataset) * 0.1)  # 10% for validation
train_size = len(full_dataset) - val_size

# Randomly split the dataset into training and validation sets
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoader instances
batch_size = 32  # You can adjust this number as needed

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
num_binary = len(binary_features.columns)  # This is based on your binary_features DataFrame
num_continuous = len(continuous_features.columns)  # This is based on your continuous_features DataFrame

# Step 3: Define the number of unique labels for your target variables.
num_genre_labels = 4  
num_key_labels = 24

# Step 4: Instantiate your model with all this information
model = MultiOutputModel(
    num_binary=num_binary, 
    num_continuous=num_continuous, 
    num_genre_labels=num_genre_labels, 
    num_key_labels=num_key_labels
)

def train_model(model, criteria, optimizer, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for binary_data, categorical_data, non_categorical_data, batch_targets in train_loader:
            optimizer.zero_grad()

            # Unpack targets
            targets_genre = batch_targets["genre"]
            targets_key = batch_targets["key"]
            targets_bpm = batch_targets["bpm"]

            # Forward pass through the model
            predictions = model(binary_data, non_categorical_data, categorical_data)

            # Calculate losses for each task
            loss_genre = criteria["genre"](predictions["genre"], targets_genre)  
            loss_key = criteria["key"](predictions["key"], targets_key)  
            loss_bpm = criteria["bpm"](predictions["bpm"], targets_bpm)

            # Combine the losses as needed, usually by summing them
            loss = loss_genre + loss_key + loss_bpm
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Print loss statistics
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# Assuming you already have the model and optimizer defined, as well as the train_loader prepared
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Define separate criteria for each task (assuming genre and key are classification, bpm is regression)
criteria = {
    "genre": nn.CrossEntropyLoss(),  # for classification tasks
    "key": nn.CrossEntropyLoss(),    # for classification tasks
    "bpm": nn.MSELoss()  # since bpm is a continuous value, we use Mean Squared Error Loss
}

# Now, you can call your training function with all necessary parameters
train_model(model, criteria, optimizer, train_loader, epochs=50)
